# Spark DataFrames Project Exercise 

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [1]:
import pyspark
spark = pyspark.sql.SparkSession.builder.appName('sess').getOrCreate()

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [2]:
df = spark.read.csv(
    "/home/jovyan/udemy_pyspark/resources/Spark_DataFrame_Project_Exercise/walmart_stock.csv",
    inferSchema=True,
    header=True,
)

# fix: Date has string type
new_column = df.Date.cast(pyspark.sql.types.DateType())
df = df.withColumn("DateTime", new_column)
df.head()

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996, DateTime=datetime.date(2012, 1, 3))

#### What are the column names?

In [3]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close', 'DateTime']

#### What does the Schema look like?

In [4]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- DateTime: date (nullable = true)



#### Print out the first 5 columns.

In [5]:
for row in df.head(5):
    print(row)

Row(Date='2012-01-03', Open=59.970001, High=61.060001, Low=59.869999, Close=60.330002, Volume=12668800, Adj Close=52.619234999999996, DateTime=datetime.date(2012, 1, 3))
Row(Date='2012-01-04', Open=60.209998999999996, High=60.349998, Low=59.470001, Close=59.709998999999996, Volume=9593300, Adj Close=52.078475, DateTime=datetime.date(2012, 1, 4))
Row(Date='2012-01-05', Open=59.349998, High=59.619999, Low=58.369999, Close=59.419998, Volume=12768200, Adj Close=51.825539, DateTime=datetime.date(2012, 1, 5))
Row(Date='2012-01-06', Open=59.419998, High=59.450001, Low=58.869999, Close=59.0, Volume=8069400, Adj Close=51.45922, DateTime=datetime.date(2012, 1, 6))
Row(Date='2012-01-09', Open=59.029999, High=59.549999, Low=58.919998, Close=59.18, Volume=6679300, Adj Close=51.616215000000004, DateTime=datetime.date(2012, 1, 9))


#### Use describe() to learn about the DataFrame.

In [6]:
desc=df.describe()
desc.show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. [Check this link for a hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.Column.cast)

If you get stuck on this, don't worry, just view the solutions.

In [7]:
import pyspark.sql.functions
import functools
_decimal = functools.partial(pyspark.sql.functions.format_number, d=2)

desc.select(
    "summary",
    _decimal(desc.Open.cast("float")).alias("Open"),
    _decimal(desc.High.cast("float")).alias("High"),
    _decimal(desc.Low.cast("float")).alias("Low"),
    _decimal(desc.Close.cast("float")).alias("Close"),
    _decimal(desc.Volume.cast("float")).alias("Volume"),
).show()

+-------+--------+--------+--------+--------+-------------+
|summary|    Open|    High|     Low|   Close|       Volume|
+-------+--------+--------+--------+--------+-------------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00|
|   mean|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|
| stddev|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|
|    min|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|
|    max|   90.80|   90.97|   89.25|   90.47|80,898,096.00|
+-------+--------+--------+--------+--------+-------------+



#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [8]:
df.select((df.High/df.Low).alias("HV ratio")).show()

+------------------+
|          HV ratio|
+------------------+
|1.0198764326019114|
|1.0147973261342302|
|1.0214151108688558|
|1.0098522508892858|
| 1.010692481693567|
|1.0123770600203459|
|1.0082994239786682|
|1.0101009760908761|
|  1.01016781935834|
|1.0099126512096774|
|1.0063704440445784|
|1.0164016736401673|
| 1.009559947570791|
|1.0077673444973507|
|1.0205761316872428|
| 1.009338138772311|
|1.0176073720585816|
|1.0095804094882654|
| 1.016072941709128|
| 1.016341993517927|
+------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [9]:
df.orderBy(df.High.desc()).head()["DateTime"]

datetime.date(2015, 1, 13)

#### What is the mean of the Close column?

In [10]:
import pyspark.sql.functions as pf
df.select(pf.mean(df.Close)).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



In [11]:
# WARNING: the following call yields something different
df.groupBy("Close").mean().head() 

Row(Close=74.5, avg(Open)=74.839996, avg(High)=74.989998, avg(Low)=74.260002, avg(Close)=74.5, avg(Volume)=8535700.0, avg(Adj Close)=66.213137)

#### What is the max and min of the Volume column?

In [12]:
df.select(
    pf.min(df.Volume),
    pf.max(df.Volume),
).show()

+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [13]:
df.filter(df["Close"]<60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [14]:
df.filter(df["High"]>80).count() / df.count()

0.09141494435612083

#### What is the Pearson correlation between High and Volume?
#### [Hint](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html#pyspark.sql.DataFrameStatFunctions.corr)

In [15]:
mean_high = df.select(pf.mean(df.High)).head()[0]
mean_volume = df.select(pf.mean(df.Volume)).head()[0]

In [16]:
top = df.select(
    pf.mean(
        (df.Volume - mean_volume) * (df.High - mean_high)
    )
).head()[0]
bottom_volume = df.select(
    pf.mean(
        (df.Volume - mean_volume)**2
    )
).head()[0]**0.5
bottom_high = df.select(
    pf.mean(
        (df.High - mean_high)**2
    )
).head()[0]**0.5
top / (bottom_volume * bottom_high)

-0.3384326061737164

In [17]:
# or use the already integrated function
df.select(
    pf.corr("High","Volume")
).head()

Row(corr(High, Volume)=-0.3384326061737161)

#### What is the max High per year?

In [22]:
(df
 .select(
     pf.year(df.DateTime).alias("Year"),
     "High",
 )
 .groupby("Year").max()
 # .select("Year","max(High)")  # if column should be disabled
 .orderBy("Year")
 .show()
)

+----+---------+---------+
|Year|max(Year)|max(High)|
+----+---------+---------+
|2012|     2012|77.599998|
|2013|     2013|81.370003|
|2014|     2014|88.089996|
|2015|     2015|90.970001|
|2016|     2016|75.190002|
+----+---------+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [21]:
(df
  .select(
     pf.month(df.DateTime).alias("Month"),
     "Close",
 )
 .groupby("Month").mean()
 .orderBy("Month")
 .show()
)

+-----+----------+-----------------+
|Month|avg(Month)|       avg(Close)|
+-----+----------+-----------------+
|    1|       1.0|71.44801958415842|
|    2|       2.0|  71.306804443299|
|    3|       3.0|71.77794377570092|
|    4|       4.0|72.97361900952382|
|    5|       5.0|72.30971688679247|
|    6|       6.0| 72.4953774245283|
|    7|       7.0|74.43971943925233|
|    8|       8.0|73.02981855454546|
|    9|       9.0|72.18411785294116|
|   10|      10.0|71.57854545454543|
|   11|      11.0| 72.1110893069307|
|   12|      12.0|72.84792478301885|
+-----+----------+-----------------+



# Great Job!